In [1]:
import sys
import polars as pl
import plotly.express as px
sys.path.insert(0, '..')
from fs_thesis import sql, show

In [2]:
df = sql("SELECT  from hosp.patients")
show(df, limit=True)

SQL query failed: Parser Error: SELECT clause without selection list


""


In [3]:
df = sql("SELECT * from hosp.admissions")
show(df, limit=True)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P49AFC,TRANSFER FROM HOSPITAL,HOME,Medicaid,English,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P784FA,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P19UTS,EMERGENCY ROOM,HOSPICE,Medicaid,English,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P06OTX,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P39NWO,EMERGENCY ROOM,None,None,English,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546023,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P13JMH,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
546024,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,EW EMER.,P38XL8,PHYSICIAN REFERRAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
546025,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P33612,EMERGENCY ROOM,DIED,Private,English,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
546026,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,EW EMER.,P036NA,EMERGENCY ROOM,HOME,Private,English,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


# SQL-Extraction for patient demographic data & time-to-event

In [4]:
df_baseline = sql("""SELECT 
    p.subject_id,
    p.gender,
    p.anchor_age,
    p.dod,
    -- Daten aus der ersten Aufnahme (Baseline)
    first_admit.admittime AS t0_time,
    first_admit.insurance,
    first_admit.language,
    first_admit.marital_status,
    first_admit.race,
    first_admit.admission_type
FROM hosp.patients p
INNER JOIN (
    -- Subquery, um nur die zeitlich erste Aufnahme pro Patient zu finden
    SELECT 
        subject_id, 
        admittime, 
        insurance, 
        language, 
        marital_status, 
        race, 
        admission_type,
        ROW_NUMBER() OVER (PARTITION BY subject_id ORDER BY admittime ASC) as row_num
    FROM hosp.admissions
) first_admit ON p.subject_id = first_admit.subject_id
WHERE first_admit.row_num = 1""")
show(df_baseline,limit=True)

,subject_id,gender,anchor_age,dod,t0_time,insurance,language,marital_status,race,admission_type
0,13585638,M,59,2195-07-25,2187-08-24 14:37:00,Private,English,MARRIED,WHITE - OTHER EUROPEAN,AMBULATORY OBSERVATION
1,13585896,F,78,NaT,2177-02-08 18:57:00,Medicare,Spanish,WIDOWED,OTHER,OBSERVATION ADMIT
2,13586115,F,19,NaT,2154-12-20 03:49:00,Private,English,SINGLE,WHITE,EU OBSERVATION
3,13588863,F,66,2152-01-09,2145-02-13 19:39:00,Medicare,English,WIDOWED,WHITE,EW EMER.
4,13589449,M,56,NaT,2161-07-01 19:11:00,Private,English,SINGLE,WHITE,OBSERVATION ADMIT
...,...,...,...,...,...,...,...,...,...,...
223447,19996366,M,65,NaT,2128-08-18 23:47:00,Medicare,Spanish,None,UNKNOWN,EW EMER.
223448,19996902,F,35,NaT,2156-09-22 01:18:00,Other,English,SINGLE,WHITE,EW EMER.
223449,19996903,M,43,NaT,2124-05-17 18:23:00,Medicaid,English,MARRIED,OTHER,EU OBSERVATION
223450,19997293,M,76,2124-02-20,2123-10-12 13:46:00,Medicare,English,SINGLE,WHITE,URGENT


In [5]:
# Zählt die eindeutigen Werte
anzahl = len(df["subject_id"].unique())
anzahl


223452

In [6]:
# Prüft auf Unique-Status
ist_unique = len(df["subject_id"].unique()) == len(df["subject_id"])
ist_unique

False

In [7]:
df_event = sql("""SELECT 
    d.subject_id,
    MIN(a.admittime) AS event_time
FROM hosp.diagnoses_icd d
JOIN hosp.admissions a ON d.hadm_id = a.hadm_id
WHERE d.icd_code LIKE 'I50%' -- Beispiel: Herzinsuffizienz
GROUP BY d.subject_id
                         """)
show(df_event, limit=True)

,subject_id,event_time
0,12662353,2160-04-29 16:03:00
1,12666222,2133-04-30 15:19:00
2,12678751,2147-07-21 22:18:00
3,12682360,2150-03-29 03:49:00
4,12683040,2139-01-08 00:03:00
...,...,...
18885,19247601,2147-05-03 11:21:00
18886,19248022,2182-08-20 14:55:00
18887,19253639,2168-12-02 23:08:00
18888,19257824,2184-11-11 02:24:00


In [12]:
# 1. Join der Tabellen
df_final = df_baseline.join(df_event, on="subject_id", how="left")

# 2. Zeitberechnungen und Target-Definition
df_final = df_final.with_columns([
    # Hilfsspalten für Differenzen in Tagen
    ((pl.col("event_time") - pl.col("t0_time")).dt.total_days()).alias("t_event"),
    ((pl.col("dod") - pl.col("t0_time")).dt.total_days()).alias("t_death"),
    
    # Event-Indikator (1 wenn Diagnose da, sonst 0)
    pl.col("event_time").is_not_null().cast(pl.Int32).alias("event_occurred")
])

# 3. Finale Duration bestimmen (Hierarchie: Event -> Tod -> Fallback)
df_final = df_final.with_columns(
    pl.coalesce([
        pl.col("t_event"),
        pl.col("t_death"),
        pl.lit(2000) # Fallback Tage
    ]).alias("duration")
)

# 4. Filter: Negative Dauern (Datenfehler) auf 0 setzen
df_final = df_final.with_columns(
    pl.col("duration").clip(lower_bound=0)
)
show(df_final, limit=True)

,subject_id,gender,anchor_age,dod,t0_time,insurance,language,marital_status,race,admission_type,event_time,t_event,t_death,event_occurred,duration
0,13585638,M,59,2195-07-25,2187-08-24 14:37:00,Private,English,MARRIED,WHITE - OTHER EUROPEAN,AMBULATORY OBSERVATION,NaT,NaN,2891.0,0,2891
1,13585896,F,78,NaT,2177-02-08 18:57:00,Medicare,Spanish,WIDOWED,OTHER,OBSERVATION ADMIT,NaT,NaN,NaN,0,2000
2,13586115,F,19,NaT,2154-12-20 03:49:00,Private,English,SINGLE,WHITE,EU OBSERVATION,NaT,NaN,NaN,0,2000
3,13588863,F,66,2152-01-09,2145-02-13 19:39:00,Medicare,English,WIDOWED,WHITE,EW EMER.,NaT,NaN,2520.0,0,2520
4,13589449,M,56,NaT,2161-07-01 19:11:00,Private,English,SINGLE,WHITE,OBSERVATION ADMIT,NaT,NaN,NaN,0,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223447,19996366,M,65,NaT,2128-08-18 23:47:00,Medicare,Spanish,None,UNKNOWN,EW EMER.,2128-08-18 23:47:00,0.0,NaN,1,0
223448,19996902,F,35,NaT,2156-09-22 01:18:00,Other,English,SINGLE,WHITE,EW EMER.,NaT,NaN,NaN,0,2000
223449,19996903,M,43,NaT,2124-05-17 18:23:00,Medicaid,English,MARRIED,OTHER,EU OBSERVATION,NaT,NaN,NaN,0,2000
223450,19997293,M,76,2124-02-20,2123-10-12 13:46:00,Medicare,English,SINGLE,WHITE,URGENT,NaT,NaN,130.0,0,130


In [13]:
df_final = df_final.with_columns(
    pl.when((pl.col("event_occurred") == 1) & (pl.col("duration") <= 365))
    .then(0) # Klasse 0: Früher Ausbruch (< 1 Jahr)
    .when((pl.col("event_occurred") == 1) & (pl.col("duration") > 365))
    .then(1) # Klasse 1: Später Ausbruch (> 1 Jahr)
    .otherwise(2) # Klasse 2: Zensiert / Kein Ereignis
    .alias("target")
)

In [14]:
# Feature Auswahl (X) und Target (y)
feature_cols = ["gender", "anchor_age", "insurance", "language", "marital_status", "race", "admission_type"]
target_col = "target"

# Konvertierung für TabPFN
X = df_final.select(feature_cols).to_pandas()
y = df_final.select(target_col).to_series().to_numpy()

In [16]:
y

array([2, 2, 2, ..., 2, 2, 2], shape=(223452,), dtype=int32)